# 1.4 Linear Regression

<a target="_blank" href="https://colab.research.google.com/github/SaajanM/mat422-homework/blob/main/1.4%20Principal%20Component%20Analysis/principal_component_analysis.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

In [ ]:
# Install a numpy package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib

In [2]:
# Import the numpy package
import numpy as np
import matplotlib
import matplotlib.pyplot as pyplt
from mpl_toolkits.mplot3d import Axes3D
import math

$\newcommand\norm[1]{\left\lVert#1\right\rVert}$

This section covers the basics of principal component analysis

## 1.4.0 Principal Component Analysis (PCA) Overview

Many problems that data scientists face are very high dimensional. However, oftentimes, we wish to preserve compute power. So we need a way to reduce the dimension of the data without losing too much detail. Principal Component Analysis (PCA) gives us such a method. What it does is to isolate the dimensions that contribute "the most" to the data and then we can disregard all "less important" dimensions.

## 1.4.1 Singular Value Decomposition

Let $A$ be an $m\times n$ matrix. Then $A^T A$ is symmetric and square and can be orthogonally diagonalized. Let $\{\mathbf{v}_1,\dots,\mathbf{v}_n\}$ be an orthonormal basis for $\mathbb{R}^n$ consisting of the eigenvectors of $A^TA$, and let $\lambda_1,\dots,\lambda_n$ be the associated eigenvalues of $A^TA$. Then for $1\leq i\leq n$,

$$
\begin{align*}
\norm{A\mathbf{v}_i}^2 &= (A\mathbf{v}_i)^TA\mathbf{v}_i\\
                       &= \mathbf{v}_i^T A^T A\mathbf{v}_i\\
                       &= \mathbf{v}_i^T(\lambda_i\mathbf{v}_i) & \text{as $\mathbf{v}_i$ is an eigenvector of $A^TA$}\\
                       &= (\mathbf{v}_i^T \mathbf{v}_i)\lambda_i & \text{commutativity of scalar multiplication over matrix multiplication}\\
                       &= \lambda_i & \text{inner product of two unit vectors is 1}
\end{align*}
$$

This necessarily means that all eigenvalues are positive (recall that $x^2$ must be positive for real $x$ and the norm is certainly real).

Then we can simply assume (by relabeling) that $\lambda_1\geq\lambda_2\geq\dots\geq\lambda_n\geq 0$.

**Definition:** We call the square roots of the eigenvalues of $A^TA$ above **singular values of $\mathbf{A}$** (denoted $\sigma_i$ for a given $\lambda_i$). They are assumed to be in decreasing order as described above.

We assert without proof that for a given $m\times n$ matrix with $r$ nonzero singular values that $\text{dim}(\text{col}(A)) = r$ (this necessitates that all other singular values are zero).

Any factorization $A=U\Sigma V^T$ is called a Singular Value Decomposition (SVD) of an $m\times n$ matrix $A$ given that $U$ is an $m\times m$ orthogonal matrix, $V$ is an $n\times n$ orthogonal matrix, and $Sigma$ is an $m\times n$ of all zeros save the main diagonal -- whose entries are the singular values of $A$.

We can compute such a decomposition of an $m\times n$ matrix $A$ as follows:
1. Obtain the $n$ eigenvalues $\lambda_i$ and $n$ eigenvectors $\mathbf{v}_i$ of $A^TA$
2. Calculate the $r$ nonzero singular values $\sigma_i$ in decreasing order.
3. Define $\mathbf{u}_i=\frac{1}{\sigma_i}A\mathbf{v}_i$ for $1\leq i\leq r$.
4. Extend the orthonormal basis for $\mathbb{R}^r$ $\mathbf{u}_1,\dots,\mathbf{u_r}$ into an orthonormal basis for $\mathbb{R}^m$ by appending unit vectors of higher dimension.
5. Then let the matrix $U = [\mathbf{u}_1\text{ }\dots\text{ }\mathbf{u}_m]$
6. Next, let the matrix $V= [\mathbf{v}_1\text{ }\dots\text{ }\mathbf{v}_n]$
7. Finally let $\Sigma$ me an $m\times n$ matrix with the nonzero singular values on the main diagonal.
8. The SVD is $A=U\Sigma V^T$

A rudimentary python implementation follows (old method implementations from previous assignments are replaced by their more performant `numpy` counterparts).

In [139]:
def svd(A):
    prod = A.T @ A  # n * n matrix
    eig = np.linalg.eig(prod)  # n eigenvalues and n eigenvectors
    eig_pairs = [
        (eig.eigenvalues[i], eig.eigenvectors.T[i])
        for i in range(eig.eigenvalues.shape[0])
    ]
    eig_pairs.sort(key=lambda v: -v[0])  # sort them descending
    for pair in eig_pairs:
        assert np.allclose(prod @ pair[1], pair[0] * pair[1])
    V = np.array(list(map(lambda v: v[1], eig_pairs)))  # get eigen vector matrix

    nz_eig_pairs = filter(lambda x: not np.allclose(x[0], 0), eig_pairs)
    nz_singles = np.array(list(map(lambda x: math.sqrt(x[0]), nz_eig_pairs)))

    # Calculate the U matrix and expand as needed
    U_small = np.array(
        [(1 / (nz_singles[i])) * (A @ V[i]) for i in range(nz_singles.shape[0])]
    )
    missing_vecs = max(U_small.shape) - min(U_small.shape)
    pad_vec_u = [(0, missing_vecs), (0, 0)]
    pad_vec_ident = [(0, missing_vecs), (0, missing_vecs)]
    U = np.pad(U_small, pad_vec_u, mode="constant") + (
        np.identity(max(U_small.shape))
        - np.pad(np.identity(min(U_small.shape)), pad_vec_ident, mode="constant")
    )

    # Calculate Sigma
    Sigma_small = np.diag(nz_singles)
    pad_vec_sigma = [
        (0, A.shape[0] - nz_singles.shape[0]),
        (0, A.shape[1] - nz_singles.shape[0]),
    ]
    Sigma = np.pad(Sigma_small, pad_vec_sigma, mode="constant")

    return (U.T, Sigma, V.T)


# A = np.array([[1, 2, 3], [4, 7, 1]])
A = np.array([[1, 4], [2, 7], [3, 1]])
print("Matrix A:\n{}".format(A))
(U, Sigma, V) = svd(A)
print("Matrix U:\n{}".format(U))
print("Matrix Sigma:\n{}".format(Sigma))
print("Transposed Matrix V:\n{}".format(V.T))
product = U @ Sigma @ V.T
print("SVD Product:\n{}".format(product))
print(
    "Is the SVD Product equal to A?: {}".format(
        "Yes" if np.allclose(A, product) else "No"
    )
)

Matrix A:
[[1 4]
 [2 7]
 [3 1]]
Matrix U:
[[-0.47902834  0.1520521   0.        ]
 [-0.84802141  0.17418982  0.        ]
 [-0.22669702 -0.97290188  1.        ]]
Matrix Sigma:
[[8.56863758 0.        ]
 [0.         2.56484894]
 [0.         0.        ]]
Transposed Matrix V:
[[-0.33321076 -0.94285237]
 [-0.94285237  0.33321076]]
SVD Product:
[[1. 4.]
 [2. 7.]
 [3. 1.]]
Is the SVD Product equal to A?: Yes


## Section 1.4.2 Low Rank Matrix Approximations

Before we begin a discussion of approximating matrices, we first need to know what it means for matrices to be "close".

**Definition: (Induced Norm)** The 2-norm of a matrix $A\in\mathbb{R}^{n\times m}$ is
$$
\norm{A}_2 = \max_{\mathbf{0}\neq\mathbf{x}\in\mathbb{R}^m}\frac{\norm{A\mathbf{x}}}{\norm{\mathbf{x}}}
           = \max_{\mathbf{0}\neq\mathbf{x}\in\mathbb{R}^m, \norm{\mathbf{x}}=1} \norm{A\mathbf{x}}
           = \max_{\mathbf{0}\neq\mathbf{x}\in\mathbb{R}^m, \norm{\mathbf{x}}=1} \mathbf{x}^TA^TA\mathbf{x}
$$

Now, we can discuss what happens when we modify our SVDs.

Let $A\in\mathbb{R}^{n\times m}$ have an SVD expressed as
$$
    A = \sum_{j=1}^{r}\sigma_j\mathbf{u}_j\mathbf{v}_j^T
$$

For $k < r$ we let
$$
    A_k = \sum_{j=1}^{k}\sigma_j\mathbf{u}_j\mathbf{v}_j^T
$$

We call $A_k$ a $k$-rank approximation of $A$. This is essentially done by restricting which singular values to use. Namely, the $r-k$ smallest singular values are discarded.

We can show that
$$
\norm{A-A_k}_2^2 = \sigma_{k+1}^2
$$

It is also known that for any rank $k$ (dimension of column space) matrix $B$, the $k-rank$ approximation defined above is the best approximation with regards to the induced norm.

A sample implementation is defined below:

In [144]:
def k_rank_approx(A, k):
    (U, Sigma, V) = svd(A)
    singulars = np.diag(Sigma)
    rank_k_singulars = singulars[0:k]
    Sigma_small = np.diag(rank_k_singulars)
    pad_vec_sigma = [
        (0, A.shape[0] - rank_k_singulars.shape[0]),
        (0, A.shape[1] - rank_k_singulars.shape[0]),
    ]
    Sigma = np.pad(Sigma_small, pad_vec_sigma, mode="constant")
    return U @ Sigma @ V.T


A = np.array([[1, 4], [2, 7], [3, 1]])
print("Matrix A:\n{}".format(A))
approx = k_rank_approx(A, 1)
print("Rank 1 Approximation:\n{}".format(approx))
(_, Sigma, _) = svd(A)
rank_2_val_squared = np.diag(Sigma)[-1] ** 2
print("Rank 2 Singular Value Squared: {}".format(rank_2_val_squared))
squared_error = np.linalg.norm(A - approx) ** 2
print("Induced Error Norm Squared(||A-A_k||_2^2): {}".format(squared_error))
print(
    "Is the square of the rank 2 singular value equal to the square of the normed error?: {}".format(
        "Yes" if np.allclose(squared_error, rank_2_val_squared) else "No"
    )
)

Matrix A:
[[1 4]
 [2 7]
 [3 1]]
Rank 1 Approximation:
[[1.36770362 3.87005091]
 [2.42123869 6.85113124]
 [0.64725678 1.83147625]]
Rank 2 Singular Value Squared: 6.578450065863193
Induced Error Norm Squared(||A-A_k||_2^2): 6.578450065863193
Is the square of the rank 2 singular value equal to the square of the normed error?: Yes


## Section 1.4.3 Principal Component Analysis

## Section 1.4.3.1 Covariance Matrix

Before we begin discussing the way PCA works and how to implement, we first need to describe what is known as the **Covariance Matrix**.

Let $[\mathbf{X}_1\dots\mathbf{X}_N]$ be a $p\times N$ matrix called the **matrix of observation**. The **sample mean** of the observation matrix vectors $\mathbf{X}_i$ is given by
$$
\mathbf{M} = \frac{1}{N}(\mathbf{X}_1+\dots+\mathbf{X}_N)
$$
For $k = 1\dots N$, let $\hat{\mathbf{X}}_k=\mathbf{X}_k - \mathbf{M}$. With this, we can construct a matrix $B = [\hat{\mathbf{X}_1}\dots\hat{\mathbf{X}_N}]$. The columns of $B$ have zero sample mean and we say $B$ is in **mean-deviation** form.

The **sample covariance matrix** of $A$ is a $p\times p$ matrix defined below.
$$
S = \frac{1}{N-1}BB^T
$$
Since any matrix of the form $BB^T$ is positive semidefinite, so is $S$. Recall, positive semidefinite just means that for every nonzero column of $S$ (denoted $z$ here) the number $z^TSz$ is non-negative.

A covariance matrix implementation is shown below.

In [202]:
def mean_dev_form(A):
    return A - np.mean(A, axis=0)


A = np.array([[1, 4], [2, 7], [3, 1]])
print("Matrix A:\n{}".format(A))
mdform = mean_dev_form(A)
print("Matrix A in Mean Deviation Form:\n{}".format(mdform))
print(
    "Is rows of MD Form Orthogonal?: {}".format(
        "Yes" if np.dot(mdform[0], mdform[1]) == 0 else "No"
    )
)

Matrix A:
[[1 4]
 [2 7]
 [3 1]]
Matrix A in Mean Deviation Form:
[[-1.  0.]
 [ 0.  3.]
 [ 1. -3.]]
Is rows of MD Form Orthogonal?: Yes


## Section 1.4.3.2 Principal Component Analysis

We now assume that the $p\times N$ data matrix $X=[\mathbf{X}_1\dots\mathbf{X}_N]$ is already in mean deviation form.

The goal of PCA is to find $k$ (where $k\leq p$) orthonormal vectors $\mathbf{v}_1,\dots,\mathbf{v}_k$ (known as the **top $\mathbf{k}$ principal components of $\mathbf{X}$**) that maximize
$$
\frac{1}{N}\sum_{i=1}^{N}\sum_{j=1}^k \langle\mathbf{X}_i\cdot \mathbf{v}_j\rangle^2
$$
where $\langle\mathbf{X}_i\cdot \mathbf{v}_j\rangle$ is the length of the projection of $\mathbf{X}_i$ onto $\mathbf{v}_j$.

We can see that for each $j$,
$$
\mathbf{v}_j^TXX^T\mathbf{v}_j = (X^T\mathbf{v}_j)^T(X^T\mathbf{v}_j) = \sum_{i=1}^{N}\langle\mathbf{X}_i\cdot \mathbf{v}_j\rangle^2
$$

So we can simplify and rephrase the variance maximization problem for each $j\leq k$ to
$$
\argmax_{\mathbf{v},\norm{\mathbf{v}} = 1} \mathbf{v}_j^TXX^T\mathbf{v}_j 
$$

We can orthogonally diagonalize $XX^T$ so that $XX^T=V\text{diag}(\lambda_1,\dots,\lambda_{p})V^T$. In view of having learned about SVD, we can then conclude that the best choices for the objective function would be the first $k$ eigenvectors corresponding to the first $k$ largest eigenvalues. These can be found in the first $k$ columns of $V$ as obtained through SVD.

Then we can perform a change of variables from each $\mathbf{x}$ to a new $\mathbf{y}$ given by $\mathbf{y} = V^T\mathbf{x}$.

To perform dimensionality reduction, simply truncate $V$ to the top $k$ PCA vectors before transposing and performing change of variables. This PCA transform  preserves most of the variance of the original data.

The percent of variance captured is given by
$$
\frac{\sum_1^k \lambda_k}{\sum_1^p \lambda_p} \times 100\%
$$

an implementation of dimensionality reduction through PCA is given below.

In [236]:
def dim_reduce(X, k):
    (_, Sigma, V) = svd(X.T / np.sqrt(X.shape[1]))
    e_vals = np.square(np.diag(Sigma))
    print(e_vals)
    reduced_evals = e_vals[0:k]
    print("Variance Kept: {}".format(np.sum(reduced_evals) / np.sum(e_vals)))
    reduced_evecs = V[:, 0:k]
    out = np.zeros((k, X.shape[1]))
    for i in range(X.shape[1]):
        out[:, i] = reduced_evecs.T @ X[:, i]
    return out


A = np.array([[1, 5], [2, 7], [3, 1]])

A = mean_dev_form(A)
print("Matrix A in Mean Deviation Form:\n{}".format(A))
res = dim_reduce(A, 2)
print("PCA dimension reduced data:\n{}".format(res))

Matrix A in Mean Deviation Form:
[[-1.          0.66666667]
 [ 0.          2.66666667]
 [ 1.         -3.33333333]]
[9.78847487 0.54485846]
Variance Kept: 1.0
PCA dimension reduced data:
[[ 0.98180399 -4.31427986]
 [ 1.01787078  0.23163764]]
